In [ ]:
import numpy as np
import pandas as pd
import gc
import keras
  
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
import sklearn
from sklearn.model_selection import train_test_split,StratifiedKFold
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE 
import cv2
from glob import glob
from skimage.transform import resize
import tensorflow as tf
import itertools
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.layers import multiply

from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, ReLU,LSTM, Bidirectional
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split

from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.regularizers import l2

from keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
import skimage
from skimage.transform import resize
import glob
import shutil
import os
import random
from PIL import Image

    
%matplotlib inline

In [ ]:
data_dir = "drive/My Drive/trainpneumo/"
test_dir =  "drive/My Drive/testpneumo/"
#valid_dir = "drive/My Drive/validK10/"
def get_data(folder):
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['normal']:
                label = 0
            elif folderName in ['pneumothorax']:
                label = 1
          #  elif folderName in ['pneumonia']:
           #     label = 2  
            else:
                label = 2
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    #img_file = skimage.transform.resize(img_file, (384,384,3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
X_data, y_data = get_data(data_dir)
X_test, y_test= get_data(test_dir)
X_valid, y_valid = get_data(valid_dir)
Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_dataHot = to_categorical(y_data, num_classes = 2)
y_testHot = to_categorical(y_test, num_classes = 2)
y_validHot = to_categorical(y_valid, num_classes=2)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(12,4),dpi=100)
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('./loss_curve.png')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.metrics import precision_recall_curve

In [ ]:
from itertools import cycle
def plot_roc(y_true,y_pred):      
    #plt.style.use('ggplot')
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes=2
    i=1
    for i in range(n_classes):

      fpr[i], tpr[i], _ = roc_curve(y_true, y_pred)
      roc_auc[i] = auc(fpr[i], tpr[i])
    colors = cycle(['blue', 'red', 'green'])
    for i, color in zip(range(n_classes), colors):
      plt.plot(fpr[i], tpr[i], color=color, lw=1.5, label='ROC curve of class {0},(area = {1:0.2f})' ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k-', lw=1.5)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    i+=1


In [ ]:
def squeeze_excite_block(input, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

In [ ]:
ROW_AXIS = 1
COL_AXIS = 2
CHANNEL_AXIS = 3
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
map_characters1 = {0: 'normal', 1: 'pneumothorax'}

In [ ]:

def conv_1_architecture(X_train,y_train,X_valid,y_valid,X_test,y_test,numclasses,numepochs,labels):
    input = Input((256,256,3))
    x = Conv2D(16, kernel_size=(3,3),padding='same')(input)
    x = BatchNormalization()
    x = ReLU()(x)
    se1 = squeeze_excite_block(x)
    x = Conv2D(16, kernel_size=(3,3),padding='same')(se1)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Conv2D(32, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se2 = squeeze_excite_block
    x = Conv2D(32, kernel_size=(3,3), padding='same')(se2)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Conv2D(64, kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se3 = squeeze_excite_block(x)
    x = Conv2D(64, kernel_size=(3,3),padding='same')(se3)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Conv2D(128, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se4 = squeeze_excite_block(x)
    x = Conv2D(128, kernel_size=(3,3), padding='same')(se4)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se5 = squeeze_excite_block(x)
    x = Conv2D(128, kernel_size=(3,3), padding='same')(se5)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Conv2D(256, kernel_size=(3,3) ,padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se6 = squeeze_excite_block(x)
    x = Conv2D(256, kernel_size=(3,3) ,padding='same')(se6)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    se7 = squeeze_excite_block(x)
    x = Conv2D(256, kernel_size=(3,3) ,padding='same')(se7)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Reshape((49, 256))(x)
    #x = Bidirectional(LSTM(256, activation="relu", return_sequences=True, trainable=False))(x)
    x = GlobalAveragePooling2D()(x)
    x= Dense(256, activation='relu')(x)
    x= BatchNormalization()(x)
    x = Dropout(0.20)(x)
    x= Dense(128, activation='relu')(x)
    x= BatchNormalization()(x)
    x = Dropout(0.20)(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=input, outputs=predictions)
    model.summary()
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(learning_rate=0.001), 
                  metrics=['accuracy'])
    weight_path="{}_weights.best.hdf5".format('cxr_reg')

    checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=4, 
                                   verbose=1, mode='min', min_delta=0.0001, cooldown=2, min_lr=1e-6)
    early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=14) # probably needs to be more patient, but kaggle time is limited
    callbacks_list = [checkpoint, early, reduceLROnPlat]

    #train_data, test_data, train_vol, test_vol = train_test_split(X_Data,y_DataHot, test_size = 0.2, shuffle=True)

    # Fit model
    history = model.fit(datagen.flow(X_train,y_train, batch_size=16), epochs=numepochs, validation_data=(X_valid,y_valid),callbacks=callbacks_list)
    # Evaluate model
    score = model.evaluate(X_test,y_test, verbose=0)
    print('Model Accuracy:', score[1], '\n')
    print('Model Loss:', score[0])
    y_validscore= model.predict(X_valid)
    print('\n', sklearn.metrics.classification_report(np.where(y_valid > 0)[1], np.argmax(y_validscore, axis=1), target_names=list(labels.values())), sep='') 
    y_pred = model.predict(X_test)
    print('\n', sklearn.metrics.classification_report(np.where(y_test > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 
    Y_pred_classes = np.argmax(y_pred,axis = 1) 
    Y_true = np.argmax(y_test,axis = 1) 
    
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
    plt.show()
    plot_learning_curve(history)
    plt.show()
    plot_roc(Y_true, Y_pred_classes)
    plt.show()
    
    return model
    


In [ ]:

import gc 
kfold = KFold(n_splits=5, shuffle=True)
for train, valid in kfold.split(X_data, y_dataHot):
  model = conv_1_architecture(X_data[train], y_dataHot[train], X_data[valid], y_dataHot[valid],X_test,y_testHot,2,65,map_characters1)
  del model
  tf.keras.backend.clear_session()
  gc.collect()